# Введение в Spark Structured Streaming 

### Streaming и batch обработки

* `batch` - обработка всех данных
* `streaming` - обработка постоянно поступающих данных
* постоянно поступать могут как структурированные, так и не структурированные данные
* желательно обрабатывать единообразно, минимальным количеством инструментов

### Как можно обрабатывать потоковые данные

(например, с помощью NiFi)

Настраиваем граф обработки, который

* периодически читает данные (файлы, таблицы)
* обрабатывает каждый элемент
* собирает в "batch" (где-то временно храня)
* записывает batch в хранилище (например, Hive)

Все это мы делаем

* другим инструментом
* на каком-то оборудовании
* каким-то образом обеспечиваем мониторинг
* каким-то образом обеспечиваем lineage

### Что предлагает Spark Structured Streaming

(начнем с конца)

* такую же обстракцию (`dataframe`) для преобразования данных
* такой же механизм получения потоковых данных (например, чтение из файлов)
* новый механизм действия (`action`): трансформация запускается при обновлении данных
* возможность работы с актуальными данными (batch + streaming) 

### Как это работает

(на примере файлов)

* создаем датафрейм из файла (также, как и в обычном случае, только `readStream`)
* настраиваем необходимые преобразования
* настраиваем параметры стриминга
    * куда записывать (файлы или kafka)
    * способ запуска алгоритмов (количество файлов или промежуток времени)
* запускаем стриминг (`start`)


### Простой пример

Шаг 1. Создаем "стриминговый" датафрейм (`readStream`), задаем - как часто выполнять (через каждые 1000 файлов)

    streamingDataFrame = spark.readStream\
        .schema(staticSchema)\
        .option("maxFilesPerTrigger", 1000)\
        .format("csv")\
        .option("header", "true")\
        .load("/data/retail-data/by-day/*.csv")

Шаг 2. Обрабатываем его так, как нам нужно (например, агрегируем)

    purchaseByCustomerPerHour = streamingDataFrame\
        .selectExpr( "CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
        .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
        .sum("total_cost")

Шаг 3. Запускаем стриминг (в данном примере будет формироваться таблица в памяти, но это могли бы быть файлы или топики Kafka)

    purchaseByCustomerPerHour.writeStream\
        .format("memory")\
        .queryName("customer_purchases")\
        .outputMode("complete")\
        .start()